# Create European Commission Awards from CORDIS

Creates European Commission awards from CORDIS bulk data. ~80K projects across FP7, H2020, and Horizon Europe.

**Prerequisites:**
- Run `scripts/local/european_commission_to_s3.py` to download and upload the data first.

**Data sources:**
- Horizon Europe (2021-2027): https://cordis.europa.eu/data/cordis-HORIZONprojects-json.zip
- Horizon 2020 (2014-2020): https://cordis.europa.eu/data/cordis-h2020projects-json.zip
- FP7 (2007-2013): https://cordis.europa.eu/data/cordis-fp7projects-json.zip

**S3 location:** `s3a://openalex-ingest/awards/european_commission/european_commission_projects.parquet`

**European Commission funder:**
- funder_id: 4320320300
- ROR: https://ror.org/00k4n6c32
- DOI: 10.13039/501100000780
- display_name: "European Commission"

**Framework Programmes:**
- FP7 (2007-2013): 25,785 projects
- H2020 (2014-2020): 35,389 projects
- HORIZON (2021-2027): 19,336 projects

**Total: 80,510 projects, EUR 166B in EC contributions**

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.european_commission_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/european_commission/european_commission_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~80K)
SELECT COUNT(*) as total_projects FROM openalex.awards.european_commission_raw;

In [ ]:
%sql
-- Check column names
DESCRIBE openalex.awards.european_commission_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.european_commission_raw LIMIT 5;

In [ ]:
%sql
-- Check framework programme distribution
SELECT framework_programme, COUNT(*) as cnt
FROM openalex.awards.european_commission_raw
GROUP BY framework_programme
ORDER BY cnt DESC;

## Step 2: Create European Commission Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.european_commission_awards
USING delta
AS
WITH
-- Get European Commission funder from OpenAlex
ec_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320320300  -- European Commission
),

awards_transformed AS (
    SELECT
        -- Generate unique ID using xxhash64 of funder_id:project_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.cordis_project_id)))) % 9000000000 as id,

        -- Display name = project title
        g.title as display_name,

        -- Description = objective
        g.objective as description,

        -- Funder info
        f.funder_id,
        g.cordis_project_id as funder_award_id,

        -- Funding amount (EC max contribution)
        TRY_CAST(g.ec_max_contribution AS DOUBLE) as amount,
        'EUR' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - map common EC schemes
        CASE
            -- Fellowships/training
            WHEN LOWER(g.funding_scheme) LIKE '%msca%' THEN 'fellowship'
            WHEN LOWER(g.funding_scheme) LIKE '%itn%' THEN 'training'
            WHEN LOWER(g.funding_scheme) LIKE '%mc-itn%' THEN 'training'
            WHEN LOWER(g.funding_scheme) LIKE '%if%' THEN 'fellowship'
            WHEN LOWER(g.funding_scheme) LIKE '%irf%' THEN 'fellowship'
            WHEN LOWER(g.funding_scheme) LIKE '%ief%' THEN 'fellowship'
            WHEN LOWER(g.funding_scheme) LIKE '%iof%' THEN 'fellowship'
            WHEN LOWER(g.funding_scheme) LIKE '%cofund%' THEN 'fellowship'
            -- ERC grants
            WHEN LOWER(g.funding_scheme) LIKE '%erc%' THEN 'research'
            WHEN LOWER(g.funding_scheme) LIKE 'horizon-erc%' THEN 'research'
            -- SME instruments
            WHEN LOWER(g.funding_scheme) LIKE '%sme%' THEN 'commercial'
            -- Research & Innovation Actions
            WHEN UPPER(g.funding_scheme) IN ('RIA', 'HORIZON-RIA') THEN 'research'
            WHEN UPPER(g.funding_scheme) IN ('IA', 'HORIZON-IA') THEN 'research'
            -- Coordination/Support
            WHEN UPPER(g.funding_scheme) IN ('CSA', 'HORIZON-CSA') THEN 'grant'
            -- Infrastructure
            WHEN LOWER(g.funding_scheme) LIKE '%infra%' THEN 'infrastructure'
            -- Collaborative projects
            WHEN UPPER(g.funding_scheme) IN ('CP', 'CP-FP', 'CP-IP') THEN 'research'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = funding_scheme (the specific EC programme)
        CONCAT(g.framework_programme, ' - ', COALESCE(g.funding_scheme, 'Unknown')) as funder_scheme,

        -- Provenance
        'cordis' as provenance,

        -- Dates
        TRY_TO_DATE(g.start_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(g.end_date, 'yyyy-MM-dd') as end_date,
        YEAR(TRY_TO_DATE(g.start_date, 'yyyy-MM-dd')) as start_year,
        YEAR(TRY_TO_DATE(g.end_date, 'yyyy-MM-dd')) as end_year,

        -- Lead investigator (coordinator organization - CORDIS doesn't provide PI names at project level)
        CASE
            WHEN g.coordinator_name IS NOT NULL THEN
                struct(
                    CAST(NULL AS STRING) as given_name,
                    CAST(NULL AS STRING) as family_name,
                    CAST(NULL AS STRING) as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.coordinator_name as name,
                        g.coordinator_country as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available at project level)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL (CORDIS project page)
        CONCAT('https://cordis.europa.eu/project/id/', g.cordis_project_id) as landing_page_url,

        -- DOI (grant DOI from CORDIS)
        g.grant_doi as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.cordis_project_id)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.european_commission_raw g
    CROSS JOIN ec_funder f
    WHERE g.cordis_project_id IS NOT NULL
      AND TRIM(g.cordis_project_id) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'cordis' AND priority = 27;

-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    27 as priority
FROM openalex.awards.european_commission_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~80K)
SELECT COUNT(*) as total_ec_awards FROM openalex.awards.european_commission_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    start_date,
    end_date
FROM openalex.awards.european_commission_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be European Commission)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.european_commission_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check framework programme distribution (via funder_scheme prefix)
SELECT 
    SPLIT(funder_scheme, ' - ')[0] as framework,
    COUNT(*) as cnt,
    ROUND(SUM(amount) / 1e9, 2) as total_eur_billions
FROM openalex.awards.european_commission_awards
GROUP BY SPLIT(funder_scheme, ' - ')[0]
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.european_commission_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (top 20 funding programmes)
SELECT funder_scheme, COUNT(*) as cnt
FROM openalex.awards.european_commission_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_description,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_coordinator,
    COUNT(doi) as has_doi,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_with_amount,
    ROUND(COUNT(description) * 100.0 / COUNT(*), 1) as pct_with_description,
    ROUND(SUM(amount) / 1e9, 2) as total_eur_billions
FROM openalex.awards.european_commission_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt
FROM openalex.awards.european_commission_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 25;

In [ ]:
%sql
-- Check coordinator countries
SELECT 
    lead_investigator.affiliation.country as country,
    COUNT(*) as cnt,
    ROUND(SUM(amount) / 1e9, 2) as total_eur_billions
FROM openalex.awards.european_commission_awards
WHERE lead_investigator.affiliation.country IS NOT NULL
GROUP BY lead_investigator.affiliation.country
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check top coordinator institutions
SELECT 
    lead_investigator.affiliation.name as institution,
    COUNT(*) as cnt,
    ROUND(SUM(amount) / 1e6, 1) as total_eur_millions
FROM openalex.awards.european_commission_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name
ORDER BY cnt DESC
LIMIT 20;